In [1]:
import pandas as pd 
import numpy as np

In [2]:
def read_goog_sp500_dataframe():
  """Returns a dataframe with the results for Google and S&P 500"""
  
  # Point to where you've stored the CSV file on your local machine
  googFile = 'data/GOOG.csv'
  spFile = 'data/SP_500.csv'

  goog = pd.read_csv(googFile, sep=",", usecols=[0,5], names=['Date','Goog'], header=0)
  sp = pd.read_csv(spFile, sep=",", usecols=[0,5], names=['Date','SP500'], header=0)

  goog['SP500'] = sp['SP500']

  # The date object is a string, format it as a date
  goog['Date'] = pd.to_datetime(goog['Date'], format='%Y-%m-%d')

  goog = goog.sort_values(['Date'], ascending=[True])

  returns = goog[[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ['float64', 'int64']]]\
            .pct_change()

  return returns

In [17]:
def read_goog_sp500_logistic_data():
  """Returns a dataframe with the results for Google and 
  S&P 500 set up for logistic regression"""
  returns = read_goog_sp500_dataframe()

  returns['Intercept'] = 1

  # Leave out the first row since it will not have a prediction for UP/DOWN
  # Leave out the last row as it will not have a value for returns
  # Resultant dataframe with the S&P500 and intercept values of all 1s
  xData = np.array(returns[["SP500", "Intercept"]][2:-1])

  yData = (returns["Goog"] > 0)[2:-1]

  return (xData, yData)

In [13]:
def read_goog_sp500_data():
  """Returns a tuple with 2 fields, the returns for Google and the S&P 500.
  Each of the returns are in the form of a 1D array"""

  returns = read_goog_sp500_dataframe()

  # Filter out the very first row which does not have any value for returns
  xData = np.array(returns["SP500"])[2:]
  yData = np.array(returns["Goog"])[2:]

  return (xData, yData)

In [14]:
def read_xom_oil_nasdaq_data():
  """Returns a tuple with 3 fields, the returns for Exxon Mobil, Nasdaq and oil prices.
  Each of the returns are in the form of a 1D array"""

  def readFile(filename):
    # Only read in the date and price at columns 0 and 5
    data = pd.read_csv(filename, sep=",", usecols=[0, 5], names=['Date', 'Price'], header=0)

    # Sort the data in ascending order of date so returns can be calculated
    data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

    data = data.sort_values(['Date'], ascending=[True])

    # Exclude the date from the percentage change calculation
    returns = data[[key for key in dict(data.dtypes) if dict(data.dtypes)[key] in ['float64', 'int64']]]\
              .pct_change()

    # Filter out the very first row which has no returns associated with it
    return np.array(returns["Price"])[1:]

  nasdaqData = readFile('data/NASDAQ.csv')
  oilData = readFile('data/USO.csv')
  xomData = readFile('data/XOM.csv')

  return (nasdaqData, oilData, xomData)

In [8]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm

/usr/local/envs/py2env/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [18]:
xData, yData = read_goog_sp500_logistic_data()

In [21]:
logit = sm.Logit(yData, xData)

In [23]:
# Fit the Logistic model
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.538306
         Iterations 6


In [24]:
# All values >0.5 predict an up day for Google
predictions = (result.predict(xData) > 0.5)

In [25]:
# Count the number of times the actual up days match the predicted up days
num_accurate_predictions = (list(yData == predictions)).count(True)

In [26]:
pctAccuracy = float(num_accurate_predictions) / float(len(predictions))

In [27]:
print "Accuracy: ", pctAccuracy

Accuracy:  0.731707317073


In [30]:
import tensorflow as tf

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [31]:
W = tf.Variable(tf.ones([1, 2]), name="W")
b = tf.Variable(tf.zeros([2]), name="b")

In [32]:
x = tf.placeholder(tf.float32, [None, 1], name="x")

In [33]:
y_ = tf.placeholder(tf.float32, [None, 2], name="y_")

y = tf.matmul(x, W) + b

In [52]:
# Logits is the affine transformation
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [35]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [36]:
# All returns in a 2D array
# [[-0.02184618]
#  [ 0.00997998]
#  [ 0.04329069]
#  [ 0.03254923]
#  [-0.01781632]]
all_xs = np.expand_dims(xData[:,0], axis=1)

In [53]:
# Another 2D array with 0 1 or 1 0 in each row
all_ys = np.array([([1,0] if yEl == True else [0,1]) for yEl in yData])

In [55]:
dataset_size = len(all_xs)

In [56]:
def trainWithMultiplePointsPerEpoch(steps, train_step, batch_size):
  init = tf.global_variables_initializer()

  with tf.Session() as sess:
    sess.run(init)

    for i in range(steps):
      if dataset_size == batch_size:
        batch_start_idx = 0
      elif dataset_size < batch_size:
        raise ValueError("dataset_size: %d, must be greater than batch_size: %d" % (dataset_size, batch_size))
      else:
        batch_start_idx = (i * batch_size) % (dataset_size)

      batch_end_idx = batch_start_idx + batch_size

      batch_xs = all_xs[batch_start_idx : batch_end_idx]
      batch_ys = all_ys[batch_start_idx : batch_end_idx]

      feed = { x: batch_xs, y_: batch_ys }

      sess.run(train_step, feed_dict=feed)

      if (i + 1) % 1000 == 0:
        print("After %d iteration:" % i)
        print(sess.run(W))
        print(sess.run(b))

        print("cross entropy: %f" % sess.run(cross_entropy, feed_dict=feed))

    # Test model
    correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))

    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    print("Accuracy: %f" % sess.run(accuracy, feed_dict={x: all_xs, y_: all_ys}))

In [57]:
trainWithMultiplePointsPerEpoch(20000, train_step, dataset_size)

After 999 iteration:
[[ 5.246492  -3.2464924]]
[ 0.13791329 -0.13791326]
cross entropy: 0.607850
After 1999 iteration:
[[ 8.041786  -6.0417876]]
[ 0.12784068 -0.12784092]
cross entropy: 0.576222
After 2999 iteration:
[[10.008763 -8.008764]]
[ 0.12074541 -0.12074568]
cross entropy: 0.560615
After 3999 iteration:
[[11.462807 -9.462808]]
[ 0.1153966  -0.11539679]
cross entropy: 0.552102
After 4999 iteration:
[[ 12.574228 -10.57423 ]]
[ 0.11123163 -0.11123197]
cross entropy: 0.547135
After 5999 iteration:
[[ 13.443815 -11.443817]]
[ 0.10792354 -0.10792376]
cross entropy: 0.544097
After 6999 iteration:
[[ 14.135796  -12.1357975]]
[ 0.10525967 -0.10525974]
cross entropy: 0.542174
After 7999 iteration:
[[ 14.693427 -12.693429]]
[ 0.1030928  -0.10309294]
cross entropy: 0.540925
After 8999 iteration:
[[ 15.14714  -13.147141]]
[ 0.10131671 -0.10131701]
cross entropy: 0.540099
After 9999 iteration:
[[ 15.519077 -13.519079]]
[ 0.09985222 -0.09985264]
cross entropy: 0.539544
After 10999 iteration:
